# Создание ценностного предложения для нового продукта.
<a id="home"></a>

На основе данных конкурирующей компании путем создания SQL-запросов требуется изучить основную информацию о книжном ассортименте и дать рекомендации для создания сервиса по торговле книгами.

## Содержание.

1. [Знакомство с данными.](#stage_1)
2. [Исследование данных.](#stage_2)
    * [Книги, вышедшие в 2020 году.](#stage_2_1)
    * [Рейтинги книг.](#stage_2_2)
    * [Издательство, выпутившее наибольшее количество книг.](#stage_2_3)
    * [Автор с самым высоким рейтингом.](#stage_2_4)
3. [Вывод и рекомендации](#stage_3)

## Знакомство с данными.
<a id="stage_1"></a>

В первую очередь, выведем основную информацию о таблицах, познакомимся с данными. Все таблицы хранятся на удаленном сервере, следовательно, перед написанием запросов к библиотеке необходимо установить соединение.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Теперь можно переходить к извлечению данных. Для начала загрузим полные данные таблиц и выведем основную информацию, используя метод info().

In [2]:
#Извлекаем необходимые данные из всех таблиц
query_books_df = '''
SELECT * FROM books 
'''
query_authors_df = '''
SELECT * FROM authors 
'''
query_publishers_df = '''
SELECT * FROM publishers 
'''
query_ratings_df = '''
SELECT * FROM ratings 
'''
query_reviews_df = '''
SELECT * FROM reviews 
'''
# Создаем БД, которыми можно оперировать в Pandas
books_df = pd.io.sql.read_sql(query_books_df, con = engine)
authors_df = pd.io.sql.read_sql(query_authors_df, con = engine)
publishers_df = pd.io.sql.read_sql(query_publishers_df, con = engine)
ratings_df = pd.io.sql.read_sql(query_ratings_df, con = engine)
reviews_df = pd.io.sql.read_sql(query_reviews_df, con = engine)

In [3]:
## Проверяем излвечение, устанавливаем основную информацию о БД:
# Присваиваем имена для БД
books_df.name = "БД books с данными о книгах"
authors_df.name = "БД authors с данными об авторах"
publishers_df.name = "БД publishers с данными об издательствах"
ratings_df.name = "БД ratings с данными о пользовательских оценках книг"
reviews_df.name = "БД reviews с данными о пользовательских обзорах книг"

# Создаем цикл для красивого вывода данных
list_of_df = [books_df,authors_df,publishers_df,ratings_df,reviews_df]
for element in list_of_df:
    print('-------------',element.name,'---------------')
    print(element.info())
    print()
    print(element.columns)
    print()
    print(element.head(2))
    print()
    print()

------------- БД books с данными о книгах ---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None

Index(['book_id', 'author_id', 'title', 'num_pages', 'publication_date',
       'publisher_id'],
      dtype='object')

   book_id  author_id                               title  num_pages  \
0        1        546                        'Salem's Lot        594   
1        2        465  1 000 Places to See Before You Die        992   

  publication_date  publisher_id  
0       2005-11-01            93  
1       2003-05-22           336  


------------- БД authors с данными об авторах ---------------
<class 'pandas.core.frame.DataFr

Данные были загружены корректно. Структура данных соответствует ТЗ, пропусков не наблюдается. Единственное, на что стоит обратить внимание при проведении запросов - тип данных в дате. Необходимо привести его к корректному виду, выполняя SQL-запрос. Можно переходить к более сложным запросам.

[К оглавлению.](#home)

## Исследование данных.
<a id="stage_2"></a>

#### Книги, вышедшие в 2020 году.
<a id="stage_2_1"></a>

Поскольку мы отчасти проверяем гипотезу о росте внимания потребителей к книгам ввиду увеличения свободного времени из-за пандемии, стоит посмотреть, что может предложить им индустрия в 2020-м году: сколько новинок представлено на рынке.

Для ответа на этот вопрос, обратимся к БД books.

In [4]:
# Создаем новый запрос:количество книг, вышедших после 1 января 2020 года
query_count_books_2000 = '''
SELECT
COUNT(book_id) AS cnt_books
FROM books
WHERE
publication_date :: date >= '2000-01-01'
'''

In [5]:
displ_query_count_books_2000 = pd.io.sql.read_sql(query_count_books_2000, con = engine)
displ_query_count_books_2000

,cnt_books
0,821


821 книга вышла после 2000 года. 

[К оглавлению.](#home)

#### Рейтинги книг.
<a id="stage_2_2"></a>

In [6]:
# Создаем новый запрос:количество обзоров и средний рейтинг всех книг.
query_ratings_reviews = '''
SELECT
books.title AS title,
COUNT(DISTINCT reviews.review_id) AS cnt_reviews,
AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
books.title
ORDER BY
cnt_reviews DESC
'''
displ_query_ratings_reviews = pd.io.sql.read_sql(query_ratings_reviews, con = engine)
displ_query_ratings_reviews

,title,cnt_reviews,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


Теперь у нас есть выборка книг, которую можно ранжировать по рейтингу и количеству обзоров. Выше представлен ТОП-5 и анти-ТОП-5 книг в зависимости от количества ревью. Стоит отметить, что не всегда самыми обсуждаемыми книгами становятся книги с высоким рейтингом. Обсуждаемость - это, скорее, заслуга маркетологов! Информация из этого рейтинга уже может быть принята к действию. 

По крайнем мере можно в бОльших количесвах заказывать книги именно популярных и хороших авторов.

[К оглавлению.](#home)

#### Издательство, выпустившие наибольшее количеств книг.
<a id="stage_2_3"></a>

Узнаем название наиболее успешного издательства в разрезе выпуска книг

In [7]:
# Создаем новый запрос:Издательство, выпустившее наибольшее количество книг толще 50 страниц
query_publisher_top = '''
SELECT
publishers.publisher AS publisher,
COUNT(books.book_id) AS cnt_books
FROM books
INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE
books.num_pages > 50
GROUP BY
publishers.publisher
ORDER BY
cnt_books DESC
LIMIT 1
'''
displ_query_publisher_top = pd.io.sql.read_sql(query_publisher_top, con = engine)
displ_query_publisher_top

,publisher,cnt_books
0,Penguin Books,42


Выпускающее наиболее количество книг издательство это "Penguin Books". Возможно экономия от масштаба позволяет издательству делать скидки для оптовых покупателей. В этом случае, наиболее популярные книги следует заказывать у этого издательства.

[К оглавлению.](#home)

#### Автор с самым высоким рейтингом.
<a id="stage_2_4"></a>

Для проработки товарного ассортимента будет полезным определить наиболее популярного автора. Создадим соответствующий запрос.

In [8]:
# Создаем новый запрос:Самый популярный автор
query_author_top = '''
SELECT
sub_query.author AS author,
AVG(sub_query.avg_rating) AS avg_rating
FROM
(SELECT
authors.author AS author,
books.title AS title,
AVG(ratings.rating) AS avg_rating,
COUNT(ratings.rating_id) AS cnt_opinions
FROM books
INNER JOIN authors ON authors.author_id = books.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
authors.author,
books.title) AS sub_query
WHERE
cnt_opinions > 50
GROUP BY
author
ORDER BY
avg_rating DESC
LIMIT 10
'''
displ_query_author_top = pd.io.sql.read_sql(query_author_top, con = engine)
displ_query_author_top

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Arthur Golden,4.107143
5,Rick Riordan,4.080645
6,William Golding,3.901408
7,J.D. Salinger,3.825581
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
9,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879


Самым популярным автором подборки с рейтингом 4,28 стала J.K. Rowling. Гарри Поттер..казалось бы, после стольких лет...=)

Это небольшое лирическое отступление вызвано искренней любовью автора к творчеству этой английской писательницы. Ее книги стали популярны во всем мире, часто переиздаются и до сих пор присутствуют в каждом книжном магазине.

In [9]:
# Создаем новый запрос:Количество ревью активных пользователей
query_avg_reviews = '''
SELECT
AVG(subquery_2.cnt_review_id) AS avg_reviews
FROM
(SELECT
username AS username,
COUNT(review_id) AS cnt_review_id
FROM
reviews
GROUP BY
username) AS subquery_2
WHERE
username IN
(SELECT
username AS username
FROM ratings
GROUP BY
username
HAVING 
COUNT(rating_id) > 50)
'''
displ_query_avg_reviews = pd.io.sql.read_sql(query_avg_reviews, con = engine)
displ_query_avg_reviews

,avg_reviews
0,24.333333


В среднем активные пользователи оставляют 17 ревью на книги. К их оценкам стоит прислушиваться при формировании каталога книг. Проведя все исследования можно сформулировать краткие рекоммендации.

[К оглавлению.](#home)

## Выводы и рекомендации.
<a id="stage_3"></a>

В качестве рекомендации можно посоветовать сделать акцент на максимально популярные книги с наибольшим количеством отзывов. При этом обновление ассортимента может быть произведено на основании тех рецензий, которые пишут наиболее активные ревьюеры. И, конечно, не забываем о Гарри Поттере в разных изданиях.

Для экономии средств, можно обращаться к крупным издательским домам с большим товарооборотом. Такие компании, возможно, смогут предоставить скидки, у них наверняка есть подходящая инфраструктура для некоторых важных бизнес-процессов.

[К оглавлению.](#home)